In [ ]:
from algorithms import td_prediction, mc_control_on_policy, sarsa, q_learning, exp_sarsa, nstep_sarsa
from env import get_random_walk_env, get_windy_gridworld_env
from policy import get_equiprobable_policy
import tqdm
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [ ]:
FIG_BASE_PATH = './figs'
if os.path.exists(FIG_BASE_PATH) is False:
    os.makedirs(FIG_BASE_PATH)

# Random Walk (7.2 RL2e Example)

Comparing n-step TD methods with varying n and $\alpha$ values. Goal is to understand how the choice of n and $\alpha$ affects the performance of the algorithm. I also compare different terminal rewards and number of states in the Random Walk environment.

In [ ]:
np.random.seed(72)
NUM_STATES = 21
env_21_negative_left = get_random_walk_env(num_states=NUM_STATES, rewards=[-1,1])
env_21_zero_left = get_random_walk_env(num_states=NUM_STATES, rewards=[0,1])
env_7_states = get_random_walk_env(num_states=7, rewards=[0,1])

def gen_episode_random_policy(env, num_episodes=10):
    policy = get_equiprobable_policy(env)
    episodes = []
    for i in range(num_episodes):
        episode = []
        state, _ = env.reset()
        done = False
        truncated = False
        while not done and not truncated:
            action, _ = policy(state)
            next_state, reward, done, truncated, _ = env.step(action)
            episode.append((state, action, reward))
            state = next_state
        episodes.append(episode)
    return episodes
    
episode_env_21_negative_left = gen_episode_random_policy(env_21_negative_left)
episode_env_21_zero_left = gen_episode_random_policy(env_21_zero_left)
episode_env_7_states = gen_episode_random_policy(env_7_states)


In [ ]:
def rmse(v, v_hat):
    return ((v - v_hat) ** 2).mean() ** 0.5

true_V_21 = np.arange(-9, 10) / 10 # derived using Bellman equation on equiprobable policy
true_V_7 = np.arange(1, 6) / 6 # derived using Bellman equation on equiprobable policy

In [ ]:
N_STEP_VALUES = [1,2,4,8,16,32,64,128,256,512]
ALPHAS = np.arange(0, 1.01, 0.01)
NUM_EXPERIMENTS = 100

def run_experiment(env, episodes, true_V, n_step_values=N_STEP_VALUES, alphas=ALPHAS, num_experiments=NUM_EXPERIMENTS, title=None, save_path=None):
    plt.figure(figsize=(20,20))
    if save_path is not None and os.path.exists(save_path):
        # load from file if exists and put in imshow and skip computation
        # assume save_path is a .png file
        img = plt.imread(save_path)
        plt.imshow(img)
        plt.axis('off')
        plt.show()
    else:
        for n in tqdm.tqdm(n_step_values):
            v_over_time = np.zeros((len(alphas), num_experiments, env.observation_space.n))
            for i, alpha in enumerate(alphas):
                for j in range(num_experiments):
                    # get dict of V and turn it into ndarray
                    V = td_prediction(env, 1, episodes, alpha, n=n)
                    V_np = np.zeros(env.observation_space.n)
                    for k, v in V.items():
                        V_np[k] = v
                    v_over_time[i, j] = V_np
            rmse_per_experiment = np.apply_along_axis(lambda x: rmse(x[1:env.unwrapped.num_states-1], true_V), 2, v_over_time)
            average_rmse = rmse_per_experiment.mean(axis=1)
            plt.plot(alphas, average_rmse, label=f'n={n}')

        plt.xlabel('$\\alpha$')
        plt.ylabel('Average RMS error over states and first 10 episodes')
        
        if title is not None:
            plt.title(title)
        plt.legend()
        if save_path is not None:
            plt.savefig(save_path)
        plt.show()    


In [ ]:
FIG_21_STATE_RANDOM_WALK_NEGATIVE_LEFT_PATH = os.path.join(FIG_BASE_PATH, '21_state_random_walk_negative_left.png')
FIG_21_STATE_RANDOM_WALK_ZERO_LEFT_PATH = os.path.join(FIG_BASE_PATH, '21_state_random_walk_zero_left.png')
FIG_7_STATE_RANDOM_WALK_PATH = os.path.join(FIG_BASE_PATH, '7_state_random_walk.png')

run_experiment(env_21_negative_left, episode_env_21_negative_left, true_V_21, title='(19 State) Random walk with (-1,1) rewards', save_path=FIG_21_STATE_RANDOM_WALK_NEGATIVE_LEFT_PATH)
run_experiment(env_21_zero_left, episode_env_21_zero_left, true_V_21, title='(19 State) Random walk with (0,1) rewards', save_path=FIG_21_STATE_RANDOM_WALK_ZERO_LEFT_PATH)
run_experiment(env_7_states, episode_env_7_states, true_V_7, title='(7 State) Random walk with (0,1) rewards', save_path=FIG_7_STATE_RANDOM_WALK_PATH)

In [ ]:
def calculate_td_update_percentage(episodes, n_values=N_STEP_VALUES):
    """
    For all n values, calculate parcentage of n-step updates that use bootstrapping vs monte carlo updates
    """
    avg_percent_td_update = np.zeros((len(n_values), len(episodes)))
    for i, n in enumerate(n_values):
        for j, episode in enumerate(episodes):
            td_update_count = 0   
            T = len(episode)
            for t in range(T):
                if t + n < T:
                    td_update_count += 1
            avg_percent_td_update[i, j] = td_update_count / (T - 1)
    
    return avg_percent_td_update.mean(axis=1)

avg_percent_td_update_21_state = calculate_td_update_percentage(episode_env_21_negative_left)
avg_percent_td_update_7_state = calculate_td_update_percentage(episode_env_7_states)

# create table of percentage of td updates that are monte carlo updates use pandas
df_21_state = pd.DataFrame({'n': N_STEP_VALUES, 'percentage_td_updates': avg_percent_td_update_21_state})
df_7_state = pd.DataFrame({'n': N_STEP_VALUES, 'percentage_td_updates': avg_percent_td_update_7_state})
df_21_state['percentage_td_updates'] = df_21_state['percentage_td_updates'].apply(lambda x: f"{x*100:.2f}%")
df_7_state['percentage_td_updates'] = df_7_state['percentage_td_updates'].apply(lambda x: f"{x*100:.2f}%")

# print table without index
print("21 State Random Walk with (-1,1) rewards\n", df_21_state.to_string(index=False))
print("7 State Random Walk with (0,1) rewards\n", df_7_state.to_string(index=False))


print(f'Episode length for 21 state random walk with (-1,1) rewards: {[len(episode) for episode in episode_env_21_negative_left]}')
print(f'Episode length for 7 state random walk with (0,1) rewards: {[len(episode) for episode in episode_env_7_states]}')

# Windy Gridworld (6.5 RL2e Example)

In [ ]:
windy_env = get_windy_gridworld_env()
NUM_STEPS = 8000
GAMMA = 0.99
EPSILON = 0.1
ALPHA = 0.5
N_STEP = 4
mc_Q, mc_episode_at_step = mc_control_on_policy(windy_env, NUM_STEPS, GAMMA, EPSILON, ALPHA, verbose=True)
sarsa_Q, sarsa_episode_at_step = sarsa(windy_env, NUM_STEPS, GAMMA, EPSILON, ALPHA, verbose=True)
exp_sarsa_Q, exp_sarsa_episode_at_step = exp_sarsa(windy_env, NUM_STEPS, GAMMA, EPSILON, ALPHA, verbose=True)
q_learning_Q, q_learning_episode_at_step = q_learning(windy_env, NUM_STEPS, GAMMA, EPSILON, ALPHA, verbose=True)
n_step_sarsa_Q, n_step_sarsa_episode_at_step = nstep_sarsa(windy_env, NUM_STEPS, GAMMA, EPSILON, ALPHA, N_STEP, verbose=True)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(mc_episode_at_step, label='MC Control')
plt.plot(sarsa_episode_at_step, label='Sarsa')
plt.plot(exp_sarsa_episode_at_step, label='Expected Sarsa')
plt.plot(q_learning_episode_at_step, label='Q Learning')
plt.plot(n_step_sarsa_episode_at_step, label=f'{N_STEP}-step Sarsa')
plt.xlabel('Episodes')
plt.ylabel('Steps per episode')
plt.title('Windy Gridworld')
plt.legend()
plt.savefig(os.path.join(FIG_BASE_PATH, 'windy_gridworld.png'))
plt.show()

In [ ]:
from collections import Counter